# **Pipline Causal Analysis - Temperature**

In [ ]:
! pip install -q datasets
! pip install -q ydata-profiling
! pip install -q transformers
! pip install -q --upgrade openai
! pip install -q huggingface_hub
! pip install scikit-learn torch langchain
! ollama pull llama2:7b
! ollama pull llama2:13b
# ! ollama pull llama2:7b-chat-q4_0
# ! ollama pull llama2:7b-chat-q5_0
# ! ollama pull llama2:7b-chat-q8_0
# ! ollama pull llama2:7b-chat-fp16
# ! ollama pull llama2:13b-chat-q4_0
# ! ollama pull llama2:13b-chat-q5_0
# ! ollama pull llama2:13b-chat-q8_0
# ! ollama pull llama2:13b-chat-fp16

In [ ]:
import pandas as pd
from datasets import load_from_disk
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import requests
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from langchain import PromptTemplate
from utils.ollamaUtil import generate_response
from utils.evaluate import calculate_cosine_similarity
from tqdm import tqdm


pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 20)
pd.set_option("max_colwidth", 1000)

Data Preparation - Take only 300 for the testing

In [ ]:
local_dataset_path = "resource/data/databricks-dolly-15k"
dataset = load_from_disk(local_dataset_path)
df = dataset.to_pandas()
test_df = df[df["category"] == "closed_qa"]
test = test_df.head(300)

In [ ]:
def remove_tags(input_string):
    start_tag = "<Answer>"
    end_tag = "</Answer>"

    if start_tag in input_string and end_tag in input_string:
        return input_string.split(start_tag, 1)[1].split(end_tag, 1)[0]
    else:
        return input_string

In [ ]:
url = "http://localhost:11434/api/generate"
instruction = """Instruction: Please strictly answer the question, but format it using custom tags like <Answer>Your answer here<Answer>. If the question cannot be answered, just answer with "I don't know". """
template = """
Instruction: {instruction}
Context: {context}
Question: {question}
"""
prompt_template = PromptTemplate.from_template(template)
columns = [
    "temperature",
    "model",
    "model_tag",
    "question",
    "context",
    "ground_truth_response",
    "prompt",
    "payload",
]
result_df = pd.DataFrame(columns=columns)

In [ ]:
# build the preprocessed csv files
for index, row in test.iterrows():
    question = row["instruction"]
    context = row["context"]
    response = row["response"]
    category = row["category"]

    prompt = prompt_template.format(
        instruction=instruction, context=context, question=question
    )

    for model in ["7b"]:
        for temperature in np.arange(0.1, 1.1, 0.3):
            model_tag = "llama2:" + model
            data = {
                "temperature": temperature,
                "model": "llama" + model,
                "model_tag": model_tag,
                "question": question,
                "context": context,
                "ground_truth_response": response,
                "prompt": str(prompt),
            }
            result_df = pd.concat([result_df, pd.DataFrame([data])], ignore_index=True)


result_df.to_excel("job_2_temperature_newtest_new.xlsx")
loaded_df = pd.read_excel("job_2_temperature_newtest_new.xlsx")

In [ ]:
# Run the experiments
csv_filename = "job_2_temperature_newtest_new.csv"
pbar = tqdm(total=len(result_df))
responses = []
similarities = []
for index, row in result_df.iterrows():
    response = remove_tags(generate_response(row))
    similarity = calculate_cosine_similarity(response, row["ground_truth_response"])
    updated_row = row.copy()
    updated_row["generated_response"] = response
    updated_row["cosine_similarity"] = similarity
    updated_dataframe = pd.DataFrame([updated_row])
    updated_dataframe.to_csv(csv_filename, index=False, mode="a", header=not index)
    pbar.update(1)
pbar.close()